In [28]:
import sys
!{sys.executable} -m pip install bs4 tqdm textblob transformers torch allennlp allennlp-models
# !{sys.executable} -m pip install bs4 tqdm textblob transformers torch==1.8.1 allennlp==2.5.0 allennlp-models==2.5.0

                                              0.0/730.2 kB ? eta -:--:--
     ---                                     61.4/730.2 kB 3.2 MB/s eta 0:00:01
     --------                               163.8/730.2 kB 2.4 MB/s eta 0:00:01
     ---------------                        297.0/730.2 kB 2.6 MB/s eta 0:00:01
     ---------------------                  419.8/730.2 kB 2.9 MB/s eta 0:00:01
     ---------------------------            522.2/730.2 kB 2.7 MB/s eta 0:00:01
     ---------------------------------      645.1/730.2 kB 2.7 MB/s eta 0:00:01
     -------------------------------------- 730.2/730.2 kB 2.7 MB/s eta 0:00:00
                                              0.0/464.5 kB ? eta -:--:--
     ---------                              112.6/464.5 kB 3.3 MB/s eta 0:00:01
     ---------------                        194.6/464.5 kB 3.0 MB/s eta 0:00:01
     ------------------------------         368.6/464.5 kB 2.9 MB/s eta 0:00:01
     -------------------------------------- 464.5/464.

  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [29 lines of output]
        Using cached setuptools-67.8.0-py3-none-any.whl (1.1 MB)
                                                    0.0/988.4 kB ? eta -:--:--
           -                                     41.0/988.4 kB 653.6 kB/s eta 0:00:02
           ----                                   112.6/988.4 kB 1.3 MB/s eta 0:00:01
           -------                                204.8/988.4 kB 1.6 MB/s eta 0:00:01
           -----------                            307.2/988.4 kB 1.7 MB/s eta 0:00:01
           --------------                         368.6/988.4 kB 1.8 MB/s eta 0:00:01
           ------------------                     481.3/988.4 kB 2.0 MB/s eta 0:00:01
           ----------------------                 573.4/988.4 kB 2.0 MB/s eta 0:00:01
           -------------------------              655.4/988.4 kB 2.1 MB/s eta 0:00:01
        

In [29]:
from typing import Union, List, Any, Optional, Dict

import os
import re
import time
import json
import glob
import pickle
import random
import urllib
import requests

from bs4 import BeautifulSoup
from tqdm import tqdm
from pathlib import Path
from textblob import TextBlob

from transformers import BertTokenizer, BertModel

import os
sys.path.append(os.getcwd())
from util import spar_utils
from util import cleaning_utils

ModuleNotFoundError: No module named 'allennlp'

#### Example IFC class and description
* source: https://search-test.bsdd.buildingsmart.org/uri/buildingsmart/ifc-4.3/class/IfcWindow


We'll use: 
* TextBlob (NLTK's implementation of punkttokenizer under the hood) to split into sentences.
* SPaR.txt to predict which objects occur, some cleaning on top of this. 


In [ ]:
label = "IfcWindow"

In [ ]:
description = """
The window is a building element that is predominately used to provide natural light and fresh air. It includes vertical opening but also horizontal opening such as skylights or light domes. It includes constructions with swinging, pivoting, sliding, or revolving panels and fixed panels. A window consists of a lining and one or several panels. A window can:be a "free standing" window, contained in an IfcSpatialElement such as an IfcBuildingStorey. fill an opening, typically in a wall. The window will then have a FillsVoids attribute which uses the IfcRelFillsElement relationship to relate the IfcWindow with the IfcOpeningElement; be part of an element assembly, typically an IfcCurtainWall. The window will then have a Decomposes attribute which uses the the IfcRelAggregates relationship to relate the window with the assembly of elements;There are two main representations for window occurrences:IfcWindow entities that have a 3D rectangle 'Profile' shape representation defined. This profile can then be used to parametrically generate the geometry of a window. If not provided, the profile of the IfcOpeningElement can be used if the window fills an opening. The parameters are specified on the relating IfcWindowType that references IfcWindowLiningProperties and IfcWindowPanelProperties for each panel in the window; IfcWindow entities that are not parametrically generated and have only 'Brep', or 'SurfaceModel' geometry.In addition, an IfcWindow may commonly include a 'FootPrint' representation defining the 2D shape of the window and its swing.the window width and height the window opening direction (by the positive y-axis of the ObjectPlacement)The IfcWindowType specifies parameters which are common to all of its occurrences of IfcWindow:the partitioning type (single panel, double panel, tripel panel, more panels) the operation type (swing, tilt and turn, pivot revolve, fixed casement, etc.) the window panel hinge side (by using two different styles for right and left opening windows) the particular attributes for the lining by the IfcWindowLiningProperties the particular attributes for the panels by the  IfcWindowPanelPropertiesREFERENCE Definition according to ISO 6707-1 Construction for closing a vertical or near vertical opening in a wall or pitched roof that will admit light and may admit fresh air. NOTE The entity IfcWindowStandardCase has been deleted. Use an IfcWindow with a 'Profile' representation instead. The IfcWindow should also have an IfcWindowType with ParameterTakesPrecedence set to 'TRUE'. IFC4 CHANGE The attributes PredefinedType and OperationType are added, the applicable type object has been changed to IfcWindowType. HISTORY New entity in IFC1.0.
"""

In [20]:
# download SPaR.txt if required
from pathlib import Path
spartxt_path = Path("SPaR.txt/")
if not spartxt_path.exists():
    !git clone https://github.com/rubenkruiper/SPaR.txt.git

Cloning into 'SPaR.txt'...
Updating files:  17% (2572/14621)
Updating files:  18% (2632/14621)
Updating files:  19% (2778/14621)
Updating files:  20% (2925/14621)
Updating files:  21% (3071/14621)
Updating files:  22% (3217/14621)
Updating files:  23% (3363/14621)
Updating files:  24% (3510/14621)
Updating files:  25% (3656/14621)
Updating files:  26% (3802/14621)
Updating files:  27% (3948/14621)
Updating files:  28% (4094/14621)
Updating files:  29% (4241/14621)
Updating files:  30% (4387/14621)
Updating files:  31% (4533/14621)
Updating files:  32% (4679/14621)
Updating files:  33% (4825/14621)
Updating files:  34% (4972/14621)
Updating files:  34% (5076/14621)
Updating files:  35% (5118/14621)
Updating files:  36% (5264/14621)
Updating files:  37% (5410/14621)
Updating files:  38% (5556/14621)
Updating files:  39% (5703/14621)
Updating files:  40% (5849/14621)
Updating files:  41% (5995/14621)
Updating files:  42% (6141/14621)
Updating files:  43% (6288/14621)
Updating files:  44% 

In [ ]:
# kind of convoluted way to import spar.txt as a module from the parent directory
import imp
with open(spartxt_path.joinpath('spar_predictor.py'), 'rb') as fp:
    spar_predictor = imp.load_module(
        'spar_predictor', fp, 'SPaR.txt.spar_predictor.py',
        ('.py', 'rb', imp.PY_SOURCE)
    )

In [ ]:
# TRAIN/LOAD
# - trains a model if needed, otherwise load from archive; 
# - best F1 on dev/validation in the paper is 80,96 trained on a GPU, CPU will be a bit lower ~77.x I think
te = spar_utils.TermExtractor(max_num_cpu_threads=1)

In [ ]:
# use existing split into sentences functionality (uses textblob)
sentences = te.split_into_sentences(description)
# first 3 sents
sentences[:3]

['The window is a building element that is predominately used to provide natural light and fresh air.',
 'It includes vertical opening but also horizontal opening such as skylights or light domes.',
 'It includes constructions with swinging, pivoting, sliding, or revolving panels and fixed panels.']

In [ ]:
objects = te.process_sentences(sentences)

In [ ]:
objects

['The window',
 'a building element',
 'natural light',
 'fresh air',
 'vertical opening',
 'horizontal opening',
 'skylights',
 'light domes',
 'constructions',
 'swinging',
 'p',
 'sliding',
 'revolving',
 'panels',
 'fixed panels',
 'A window',
 'a lining',
 'one',
 'panels',
 'A window',
 'a',
 'free standing " window',
 'an IfcSpatialElement',
 'an IfcBuildingStorey',
 'an opening',
 'a wall',
 'The window',
 'a FillsVoids attribute',
 'the IfcRelFillsElement relationship',
 'the IfcWindow',
 'the IfcOpeningElement',
 'an element assembly',
 'an IfcCurtainWall',
 'The window',
 'a Decomposes attribute',
 'the',
 'the IfcRelAggregates relationship',
 'the window',
 'the assembly',
 'elements',
 'representations',
 'window occurrences',
 'IfcWindow entities',
 'a 3D rectangle',
 'Profile shape representation',
 'profile',
 'the geometry',
 'a window',
 'the profile',
 'the IfcOpeningElement',
 'the window',
 'an opening',
 'The parameters',
 'the relating IfcWindowType',
 'IfcWindow

Get all the names and definitions from the bSDD

In [43]:
import requests
import json
import pandas as pd
from pprint import pprint

query = """query MyQuery {
	classification(limit: "50"){
    id name
    definition
    synonym
  }
}"""

url = 'https://bsdd.ontotext.com/graphql/'
r = requests.post(url, json={'query': query})
print(r.status_code)

json_data = json.loads(r.text)
pprint(json_data['data']['classification'])
# pprint(r.text)

200
[{'definition': 'Revêtement de sol résilient de type LVT à poisser, en dalles '
                "et lames, constitué d'une couche d'usage et de plusieurs "
                'couches de composition différentes ayant des propriétés '
                'acoustiques, respectant les classements U2s, U3 et U4  du '
                'classement des locaux',
  'id': 'https://identifier.buildingsmart.org/uri/ATALANE/REX-OBJ-1.0/class/262c5f4a-a631-4041-a5ea-569ebad4265b',
  'name': 'Revêtement de sol résilient LVT acoustique à poisser',
  'synonym': []},
 {'definition': 'Caractérisation du sol',
  'id': 'https://identifier.buildingsmart.org/uri/ATALANE/REX-OBJ-1.0/class/587d5543-7b55-4475-a831-694dbebfd7aa',
  'name': 'Caractérisation du sol',
  'synonym': []},
 {'definition': 'Typologie',
  'id': 'https://identifier.buildingsmart.org/uri/ATALANE/REX-OBJ-1.0/class/a3fe4219-3927-4dc5-81b1-45e12e213528',
  'name': 'Typologie',
  'synonym': []},
 {'definition': "Ensemble des propriétés d'épaisseur